# Sensor Data Fusion

Lecturer: Prof. Baum \
Tutor: Shishan Yang and Laura Wolf \
Semester: Winter 20/21

## Homework 3

Look again at the GPS setting of last homework (sattelites are $20200 \text{km}$ above mean sea level, each  satellite broadcasts its location in spherical coordinates $[\theta, \phi, r]^T$ plus the emission time (see figures below), assume that the speed-of-light is $c=3\cdot10^8\text{ms}^{-1}$ and the Earth is an ideal sphere with radius $r^{\text{E}}=6370 \text{km}$)

This time, the device only receives the first three sattelite signals at time $t=0\text{s}$:

| $i$ | $p_i$ | $t_i$ |
| :--- | :---: | ---: |
| 1 | $[  0^{°},  40^{°}, 20200 \text{km}]^T$ | -67.603 ms |
| 2 | $[ 10^{°},  20^{°}, 20200 \text{km}]^T$ | -70.102 ms |
| 3 | $[ 10^{°}, -10^{°}, 20200 \text{km}]^T$ | -78.690 ms |


![Satellite Visuals](https://owncloud.gwdg.de/index.php/s/guLnIkLe4aOcL1E/download)

---
The following tasks will have missing sections marked that you should fill out. 

Missing code parts are marked by
```
# ... code code code
=== YOUR CODE HERE ===

=== END OF YOUR CODE ===
# ... code code code
```
If you are asked to implement a function, make sure to check what variable will be returned by the function and to fill it accordingly. Do not change code outside of the indicated sections.

Furthermore, some questions require theoretical answers instead of python code.

Such questions will have a field marked like this: 

=== YOUR ANSWER HERE === 

In [165]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy.linalg import inv
import sympy as sym

In [166]:
# Definition of global variables
speed_light = 3 * math.pow(10, 5)  # Speed of light def. - 10^5 because we want km/s
R = 6370  # earth radius (assuming earth is a perfect sphere)
t = np.array([67.603e-3, 70.102e-3, 78.690e-3])  # emission times

# location in spherical coordinates
p = [
    [0, 40, 20200],
    [10, 20, 20200],
    [10, -10, 20200]
    ]
p = np.array(p)
spher_coor_p = p  # add a second alias...

# Ground Truth target location from the last exercise
gt = np.array([4390.2170784, 589.75165586, 4584.055838])

In [167]:
# You will also be provided with sphere_to_cartesian and cartesian_to_sphere functions
def sphere_to_cartesian(p_sphere, R):
    """
    Function that converts spherical to cartesian coordinates. 
    :param p_sphere: 3x1 vector [theta, phi, R] with
    theta, longitude in degree
    phi, latitude in degree
    r, altitude in km
    :param R: earth radius in km
    :return: p_cartesian, converted cartesian coordinates, as 3x1 vector [x,y,z]
    """
    # make sure everything is numpy
    p_sphere = np.array(p_sphere)
    
    # unbox p_sphere and convert to radians
    theta = math.radians(p_sphere[0])
    phi = math.radians(p_sphere[1])
    r = p_sphere[2]
    
    # calculate x,y,z, position
    x = (R + r) * math.cos(phi) * math.cos(theta)
    y = (R + r) * math.cos(phi) * math.sin(theta)
    z = (R + r) * math.sin(phi)
    p_cartesian = [x, y, z]
    
    return p_cartesian

def cartesian_to_sphere(p_cartesian, R):
    """
    :param p_cartesian: Point represented in cartesian coordinates as 3x1 vector
    :param R: Earth Radius in km
    :return: p_sphere, point represented in sphere coordinates, 
            3x1 vector: [theta, phi,r],  with
            theta, longitude in degree
            phi, latitude in degree
            r, altitude in km
    """
    p_cartesian = np.array(p_cartesian)
    
    x,y,z = p_cartesian  # unpack array
    r = math.sqrt(x**2 + y**2 + z**2) - R
    theta = math.degrees(np.arctan(y/x))
    phi = math.degrees(np.arctan(z / (math.sqrt(x**2 + y**2))))
    p_sphere = [theta, phi, r]
    
    return p_sphere

In [168]:
# Further variable definitions:

cart_coor_p = np.array([sphere_to_cartesian(pos, R) for pos in spher_coor_p])

d = speed_light * t

---
### a)
Would the reformulation from last time still work? If not, explain why.

=== YOUR ANSWER HERE ===

---
### b)
Instead of reformulating the measurement equation into a linear equation, implement the Gauss-Newton method to calculate the least squares solution.

Hints: 
- you will need to use symbolic python (the package sympy is already imported)
- you will need to use sym.Matrix instead of np.array for most variables: convert np arrays into sympy as necessary, the two packages don't mix well
- sympy provides .jacobian(...) for calculating the Jacobian
- sympy provides .subs(...) for symbolic substitution 


In [169]:
def gauss_newton(y, h, x1, x2, x3, x_init):
    """
    Applies the Gauss-Newton method.
    :param y: measurements
    :param h: non-linear measurement matrix
    :param x1: symbolic variable representing the dimension of the state
    :param x2: symbolic variable representing the dimension of the state
    :param x3: symbolic variable representing the dimension of the state
    :return: x_hat, the resulting estimate
    """
    y = sym.Matrix(3, 1, y)
    x_init = np.array(x_init)

    # === YOUR CODE HERE ===

    print('h: ', h)
    X = sym.Matrix(3, 1, [x1, x2, x3])
    x_hat = sym.Matrix(3, 1, x_init)
    r_xl = y - h
    J = r_xl.jacobian(X)

    delta = 1
    min_delta = 1e-4

    while (delta > min_delta):
        x_hat_l = [
            (x1, x_hat[0]),
            (x2, x_hat[1]),
            (x3, x_hat[2]),
        ]
        J_l = J.subs(x_hat_l)
        r_xhat_l = y - h.subs(x_hat_l)

        x_hat_new = x_hat - (J_l.T @ J_l).inv() @ J_l.T @ r_xhat_l
        delta = sym.Matrix.norm(x_hat_new - x_hat)
        x_hat = x_hat_new

    # === END OF YOUR CODE ===

    return x_hat

In [170]:
# First, we create the symbolic variables
x1 = sym.symbols("x1")
x2 = sym.symbols("x2")
x3 = sym.symbols("x3")

x = sym.Matrix(3, 1, [x1, x2, x3])

# FYI: http://omz-software.com/pythonista/sympy/modules/matrices/matrices.html
# if you want a deeper understanding of why lambda functions are used in the def. of h
h = [
    sym.sqrt(np.sum(sym.Matrix(1, 3, lambda i,j: (cart_coor_p[0,j] - x[j,0])**2).T)),
    sym.sqrt(np.sum(sym.Matrix(1, 3, lambda i,j: (cart_coor_p[1,j] - x[j,0])**2).T)),
    sym.sqrt(np.sum(sym.Matrix(1, 3, lambda i,j: (cart_coor_p[2,j] - x[j,0])**2).T))
]

h = sym.Matrix(3, 1, h)
# Next, we apply the gauss newton algorithm
x_init = np.array([0, 0, 0])
x_hat_gn = gauss_newton(d,h,x1,x2,x3,x_init)

x_hat_gn = np.reshape(np.array(x_hat_gn), (3,)).astype(float)

print("\n---\nGN Solution after using {} as initial guess:".format(x_init))
print(x_hat_gn)
print("\nDistance to true solution: {:.4f}".format(np.linalg.norm(gt-x_hat_gn)))

h:  Matrix([[20353.8008536712*sqrt(2.41384265852573e-9*x2**2 + (1 - 4.9130872763729e-5*x1)**2 + 0.704088191041847*(1 - 5.8551894123463e-5*x3)**2)], [24588.3184880435*sqrt((1 - 4.06697188539456e-5*x1)**2 + 0.0310912041257634*(1 - 0.000230649437145345*x2)**2 + 0.136593117911764*(1 - 0.000110041565681712*x3)**2)], [25768.8164671408*sqrt((1 - 3.8806594058177e-5*x1)**2 + 0.0310912041257634*(1 - 0.000220083131363424*x2)**2 + 0.0320578670997533*(-0.000216739574073904*x3 - 1)**2)]])

---
GN Solution after using [0 0 0] as initial guess:
[4389.90660512  590.01290096 4584.03305667]

Distance to true solution: 0.4064


---
### c)
Alternatively, use the Bancroft solution to determine the position of the GPS device.

Hint: 
- numpy provides np.roots() to find the roots of a polynomial

__Solution (Theory):__
$\newcommand\vect[1]{\begin{bmatrix}#1\end{bmatrix}}$

Squared meas. equation:

$
d_i^2 = ||x - p_i ||^2+ e^*_i$

$=  (x_1-p_{i,1})^2  + (x_2-p_{i,2})^2 + e^*_i$

$=   -2x_1 p_{i,1}  - 2x_2 p_{i,2}  + ||p_i||^2 +R^2   + e^*_i$

with   $R^2:= ||x||^2= (x_1)^2 + (x_2)^2$


Linear measurement equation for given $R^2$:
$$y =  \textbf{H}_1 x   + \textbf{H}_2 R^2 + e^* $$
with
$$
y  =  \vect{d_1^2 - ||p_1 ||^2 \\ \vdots \\ d_m^2 - ||p_m ||^2 } \enspace, \enspace   \textbf{H}_1  =  \vect{ -2 p_{i,1}  & -2 p_{i,2}      \\  \vdots & \vdots  \\   -2 p_{m,1}  & -2 p_{m,2} } \enspace , \enspace  \textbf{H}_2  =  \vect{ 1      \\  \vdots \\   1}
$$



Least squares solution for a fixed $R^2$:

\begin{eqnarray*}
x^{LS}(R^2)  &=& (\textbf{H}_1^\text{T}  \textbf{H}_1)^{-1} \textbf{H}_1^\text{T}  (y- \textbf{H}_2 R^2) \\  
                &= &  z_1 + R^2   z_2 
\end{eqnarray*}

with $z_1 := (\textbf{H}_1^\text{T}  \textbf{H}_1)^{-1} \textbf{H}_1^\text{T} y$ and $z_2:= -(\textbf{H}_1^\text{T}  \textbf{H}_1)^{-1} \textbf{H}_1^\text{T}   \textbf{H}_2 $ 


What is $R^2$? 

\begin{eqnarray*}
R^2 &=& ||x^{LS}(R^2)||^2 \\
    & =&  (z_1 + R^2 z_2)^\text{T}  \cdot (z_1 + R^2 z_2)  
\end{eqnarray*}


Solve the following quadratic equation for $R^2$:

\begin{equation*}
0 = z_1^\text{T}  z_1  + z_1^\text{T}  z_2  R^2  + R^2 z_2^\text{T}  z_1  +  (R^2)^2 z_2^\text{T}  z_2   -R^2 
\end{equation*}


In [171]:
def bancroft(p, d):
    """
    Calculates the bancroft solution.
    :param p: The cartesian coordinates with shape (3x3)
    :param d: The distances between satellites and receiver with shape (3,)
    :return: x_hat, the position estimate
    """
    # ensure everything is numpy
    p = np.array(p)
    d = np.array(d)
    
    # === YOUR CODE HERE ===
    
    y = d**2 - np.linalg.norm(p, axis=1)**2
    H_1 = -2 * p.T
    H_2 = np.ones(y.shape[0])

    z_1 = np.linalg.inv(H_1.T @ H_1) @ H_1.T @ y
    z_2 = -np.linalg.inv(H_1.T @ H_1) @ H_1.T @ H_2
    print('z_1: ', z_2.shape)

    

In [ ]:
)
    
    # === END OF YOUR CODE ===
    
    return x_hat

In [172]:
# Now, the bancroft solution will be applied to find an estimate
x_hat_b = bancroft(cart_coor_p, d)
print("Bancroft solution:")
print(x_hat_b)
print("\nDistance to true solution: {:.4f}".format(np.linalg.norm(gt-x_hat_b)))

z_1:  (3,)


NameError: name 'x_hat' is not defined